#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Define plot saving function

In [2]:
# Setting up default settings for plots, rc = Runtime configuration

plt.rc('figure',figsize = (10,6))          
plt.rc('font',size =14)                    # Defining font size for all texts (unless overriden)
plt.rc('xtick', labelsize = 14)            # you know what tick and tick_label is right?
plt.rc('ytick', labelsize = 14)
plt.rc('axes',labelsize= 14, titlesize= 16, titleweight = 'bold')   # Defining config of Xaxis, Yaxis and title

#### Get the dataset

In [8]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784',version=1)
X,y = np.array(mnist['data']), np.array(mnist['target'])
X.shape,y.shape

((70000, 784), (70000,))

#### Train-Test split

In [9]:
# Train-Test Split
X_train,X_test,y_train,y_test = X[:60000],X[60000:],y[:60000],y[60000:]

# _Multiclass Classification/Multinomial Classification_

There are two ways in which we can perform multiclass classification 
1. Use multiclass classifiers
2. Use multiple binary classifiers

There are two ways to use binary classifiers for multiclass classification
1. OvR (One vs Rest/One vs All) 
    - We train one binary classifier for each class
    - Each classifier learn to detect "that class" or "not that class"
    - During prediction, all the classifiers run, one with the highest score decides the final class

    eg. In this dataset, we will train 10 binary classifier for all 10 classes(0-9), 1st classifier give score for any image to be "0"or"Not 0", 1st will give score for "1"or"not 1" and so on...

2. OvO (One vs One)
    - We train one binary classifier for each pair of classes
    - For _n_ number of classes we get <sup>n</sup>C<sub>2</sub> = $\frac{(n)*(n-1)}{2}$
    - During prediction, each model gives its vote and the class with most number of votes is selected

    eg. In this dataset, we will train 10*9/2 = 45 binary classifiers that will vote to give final output (predict class), 1st classifier is trained on "0"or"1", 2nd on "0"or"2" and so on....
